In [1]:
import re

#system prompt
with open('resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    sys_prompt = f.read()

#initial instruction
with open('resources/initial_prompts/simple_prompt_v2.txt', 'r') as f:
    content = f.read()

prefix = "MY MOVE:"
def parse_reply(text, prefix=prefix):
    parsed = text.split(prefix)
    return parsed[1]
    
    #if not text.startswith(prefix):
    #    return False
    
    #return text[len(prefix):]

def generate_board_simple(n):
    
    board = "[start]"
    
    for i in range(n):
        board += " []"
    board += " [end]"
    
    return board

def check_move(current_state, response, roll):
    current_state_list = current_state.split() #parse the fields into individual elements
    new_state_list = response.split()
    
    current_state_token_inx = current_state_list.index('[A:X]')
    new_state_token_inx = new_state_list.index('[A:X]')
    
    if current_state_token_inx + roll == new_state_token_inx:
        return True
    
    else:
        return False
    
    
def add_message(context, msg, role='user', sysmsg = sys_prompt):
    if context == []:
        context = [
            {"role": "system", "content": sysmsg}
        ]
    context.append({"role": role, "content": msg})
    return context


#game class
class Game():
    def __init__(self, lmm, initial_msg, sys_prompt):
        self.msg = initial_msg #initial message -> contains game instructions
        self.sys_prompt = sys_prompt #system prompt
        self.lmm = lmm #language model object
        self.rolls = [3,6,4,5,2,6,4,1] #list of rolls per turn
        self.board = "□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □"
        self.turn = 0 #turn counter
        self.context = [] #memorized context
        self.current_state = '' #current state of the board
        self.prefix = "MY MOVE:" #used to parse the response
        self.token_out = False #false if no token on board yet.

    #makes move
    def make_move(self):
        if self.turn == 0: #first turn prompt is slightly different
            in_m = f"Are you ready to play?\nBeginning State: {self.board}\n Turn num: {self.turn}, Roll: {self.rolls[self.turn]}. Where will you move your token? Let's think step by step."
            self.add_message(self.msg + '\n' + in_m)
            init_prompt, init_resp, init_resp_text = self.lmm.generate_response(self.context)
            
            #needs try catch shit here, and everywhere we actually parse/split/index/etc.
            parsed = self.parse_reply(init_resp_text)
            
            #check validity of the move
            if self.check_move(parsed, self.rolls[self.turn]) == True:
                print(self.current_state)
                print(parsed)
                self.current_state = parsed 
                self.add_message(init_resp_text, role = 'assistant' ) #if valid, adds response to context
                self.turn+=1 #updates step
                if 'X' in self.current_state: #if token has been placed on the board
                    self.token_out = True

                
            else:
                print(f'fail at turn: {self.turn}')
                print(f'roll: {self.rolls[self.turn]}')
                print(f'{init_resp_text}')
        else:
            #if not first turn
            m = f"Next turn number: {self.turn}, Roll: {self.rolls[self.turn]}.\n Where will you move your token? Let's think step by step."
            self.add_message(m, role = 'user') 
            given_prompt, response, response_text = lmm.generate_response(self.context)
            
            parsed = self.parse_reply(response_text)
            if  self.check_move(parsed, self.rolls[self.turn]) == True:
                print(self.current_state)
                print(parsed)
                self.current_state = parsed
                self.add_message(response_text, role ='assistant')
                self.turn+=1
                if 'X' in self.current_state: #if token has been placed on the board
                    self.token_out = True

            else: 
                print(f'fail at turn: {self.turn}')
                print(f'roll: {self.rolls[self.turn]}')

                print(f'{response_text}')
            
    
    def add_message(self, message, role='user'):
        #add message to context
        if self.context == []:
            self.context = [
                {"role": "system", "content": self.sys_prompt}
            ]
        self.context.append({"role": role, "content": message})
    
    
    
    def check_move(self, response, roll):
        #move checker, checks validity
        if self.token_out == False:     #if the token hasn't been added to the board
            if roll != 6: 
                #can't add token if 6 isn't roleld
                for item in response.split():
                    if 'X' in item:
                        return False
                    else:
                        return True
            if roll == 6:
                #has to add token if 6 is rolled
                if 'X' in response.split()[0]:
                    return True
                else:
                    return False
        else:
            #counts the number of steps moved. Looks at the difference between the current state of the board and LLM's choice of movement. 
            current_state_list = self.current_state.split() #parse the fields into individual elements
            new_state_list = response.split()
            if len(current_state_list) != len(new_state_list):
                print('field count err')
                return False
            else:
            
                if self.token_out == True:
                    current_state_token_inx = current_state_list.index('X') + 1
                    if 'X' in response:
                        new_state_token_inx = new_state_list.index('X') + 1
                        print(current_state_token_inx, roll, new_state_token_inx)
                        if current_state_token_inx + roll == new_state_token_inx:
                            return True
                        else:
                            return False
                    else:
                        return False                                        

    def parse_reply(self, text):
        #parses the reply so we only take out the board
        parsed = text.split(self.prefix)
        return parsed[1]
    
    def split(self, text):   
        #regex to split the board into a list
        parts = re.findall(r'\[.*?\]', text)
        return parts
       
    def reset(self):
        #resets game state
        self.turn = 0
        self.context = []

In [2]:
from backends import ModelSpec, Model, get_model_for, load_model_registry
#need to add api key
#need to add model_name in the dict
#need to create key.json
load_model_registry()  # load default model registry from backends folder
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106") # works without registered entry when openai_api.py backend is available


In [ ]:
lmm: Model = get_model_for(THIS_MODEL)

lmm.set_gen_args(temperature=0.0, max_tokens=400)


In [ ]:
instance = Game(lmm, content, sys_prompt)

In [ ]:
instance.make_move()